In [55]:
#Imports
import pandas as pd
import alpaca_trade_api as tradeapi
from pathlib import Path
import os
from dotenv import load_dotenv

In [56]:
# Load the environment variables from the .env file by calling the load_dotenv function
load_dotenv()

True

In [57]:
# Set the variables for the Alpaca API and secret keys
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

# The Alpaca tradeapi.REST object
alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version="v2")

In [58]:
#Set Tickers for stocks and crypto for analysis only looking at top 10 existing for at least 4 years
stock_tickers = ['MSFT','AAPL','TSLA','AMZN','NVDA','GOOG','FB','ADBE','NFLX','PYPL']
crypto_tickers = ['BTC','ETH','BNB','ADA','XRP','DOGE','LTC','LINK','BCH','XLM']

In [59]:
# Setting time frame to 1D
timeframe = "1D"

# Set the start/ end datetime of 5 years from today 
start_date = pd.Timestamp('2018-02-01', tz='America/New_York').isoformat()
end_date = pd.Timestamp('2021-04-27',tz='America/New_York').isoformat()

# Set limit of rows to the max to maximaize
limit_rows = 1000

# Get the 2 years data frame for stocks
stocks_df = alpaca.get_barset(
    stock_tickers,
    timeframe,
    start = start_date,
    end = end_date,
    limit = limit_rows).df

# Visualazing the data 
stocks_df.head()

AAPL                                      \
                              open    high       low   close    volume   
time                                                                     
2018-02-01 00:00:00-05:00  167.165  168.62  166.7600  167.70  29765526   
2018-02-02 00:00:00-05:00  166.470  166.80  160.1000  160.41  67919941   
2018-02-05 00:00:00-05:00  159.100  163.88  156.0000  156.49  59470254   
2018-02-06 00:00:00-05:00  154.910  163.72  154.0000  163.06  54780528   
2018-02-07 00:00:00-05:00  163.085  163.40  159.0685  159.52  41035232   

                             ADBE                                       ...  \
                             open    high       low   close     volume  ...   
time                                                                    ...   
2018-02-01 00:00:00-05:00  199.12  201.75  198.0845  199.38  1532011.0  ...   
2018-02-02 00:00:00-05:00  197.33  199.40  195.4400  195.63  1410466.0  ...   
2018-02-05 00:00:00-05:00  194.06  198.46  188.0000  190.27  2332614.0  ...   
2018-02-06 00:00:00-05:00  186.59  194.82  182.1000  194.52  3130162.0  ...   
2018-02-07 00:00:00-05:00  193.87  196.24  192.0200  192.36  1714797.0  ...   

                            PYPL                                    TSLA  \
                            open    high    low  close    volume    open   
time                                                                       
2018-02-01 00:00:00-05:00  79.96  80.950  76.70  78.40  37352330  351.00   
2018-02-02 00:00:00-05:00  78.01  79.620  76.28  76.55  19063799  348.44   
2018-02-05 00:00:00-05:00  75.99  78.980  74.68  74.70  15685427  337.97   
2018-02-06 00:00:00-05:00  72.51  76.515  72.25  75.68  16194538  325.21   
2018-02-07 00:00:00-05:00  75.35  77.580  74.94  75.59   9497980  338.83   

                                                            
                             high     low   close   volume  
time                                                        
2018-02-01 00:00:00-05:00  359.66  348.63  349.25  3065835  
2018-02-02 00:00:00-05:00  351.95  340.51  343.75  2730271  
2018-02-05 00:00:00-05:00  344.47  333.00  333.15  3124163  
2018-02-06 00:00:00-05:00  336.22  323.50  333.97  3703402  
2018-02-07 00:00:00-05:00  346.00  335.66  344.77  4422730  

[5 rows x 50 columns]

In [60]:
#Create for loop to read in Change % data and clean column of string values and convert to float /100
crypto_daily_df = pd.DataFrame()
for crypto in crypto_tickers:
    df = pd.read_csv(Path(f"Crypto_Historical_Data/{crypto}.csv"), index_col="Date", parse_dates=True, infer_datetime_format=True)
    #If dtype is O ---object then str.replace
    if df['Change %'].dtypes=='O':
        df['Change %'] = df['Change %'].str.replace('%','')
    else:
        df['Change %'] = df['Change %'].replace('%','')
    crypto_daily_df[f'{crypto}'] = df['Change %'].astype('float').divide(100)

#drop na values or change to 0 depending on need
crypto_daily_df = crypto_daily_df.dropna()

display(crypto_daily_df.head())
display(crypto_daily_df.tail())

,BTC,ETH,BNB,ADA,XRP,DOGE,LTC,LINK,BCH,XLM
Date,,,,,,,,,,
2021-04-25,-0.0225,0.0454,0.0137,-0.0111,-0.0155,-0.0700,0.0047,0.0236,-0.0185,-0.0211
2021-04-24,-0.0206,-0.0639,-0.0552,-0.0473,-0.1026,0.0856,-0.0671,-0.1119,-0.0800,-0.0560
2021-04-23,-0.0113,-0.0141,0.0410,0.0092,0.0098,-0.0463,-0.0458,-0.0029,-0.0177,0.0008
2021-04-22,-0.0388,0.0171,-0.0690,-0.0483,-0.1020,-0.1467,-0.0189,-0.0288,-0.0610,-0.0840
2021-04-21,-0.0471,0.0123,-0.0752,-0.0496,-0.0690,-0.0380,-0.0119,-0.0697,-0.0330,-0.0708


,BTC,ETH,BNB,ADA,XRP,DOGE,LTC,LINK,BCH,XLM
Date,,,,,,,,,,
2018-02-06,0.1099,0.1248,0.2307,0.1325,0.1182,0.2460,0.1351,0.0622,0.0930,0.1224
2018-02-05,-0.1561,-0.1598,-0.2025,-0.1702,-0.1670,-0.1958,-0.1528,-0.0882,-0.2379,-0.1379
2018-02-04,-0.1103,-0.1462,-0.1387,-0.1767,-0.1478,-0.1437,-0.0813,-0.1426,-0.0905,-0.1611
2018-02-03,0.0391,0.0550,0.0000,0.1600,0.0673,0.1279,0.2230,0.0348,0.0677,0.0783
2018-02-02,-0.0314,-0.1090,0.0247,-0.0515,-0.0547,-0.1093,-0.0765,-0.0410,-0.0639,-0.0859


In [61]:
#check for nan values and drop
stocks_df.isna().sum()
stocks_df = stocks_df.dropna()

In [62]:
#Calculate the daily change for the stocks_df using for loop
stocks_daily_df = pd.DataFrame()
for stocks in stock_tickers:
    stocks_daily_df[f'{stocks}'] = stocks_df[stocks]['close'].pct_change().dropna()

display(stocks_daily_df.head())
display(stocks_daily_df.tail())

,MSFT,AAPL,TSLA,AMZN,NVDA,GOOG,FB,ADBE,NFLX,PYPL
time,,,,,,,,,,
2018-02-02 00:00:00-05:00,-0.026316,-0.043470,-0.015748,0.029639,-0.028984,-0.047164,-0.014528,-0.018808,0.008752,-0.023597
2018-02-05 00:00:00-05:00,-0.040976,-0.024437,-0.030836,-0.027884,-0.084964,-0.051531,-0.047379,-0.027399,-0.049067,-0.024167
2018-02-06 00:00:00-05:00,0.037955,0.041984,0.002461,0.038014,0.055647,0.025392,0.023006,0.022337,0.045149,0.013119
2018-02-07 00:00:00-05:00,-0.019269,-0.021710,0.032338,-0.019191,0.014409,-0.030210,-0.028366,-0.011104,-0.004256,-0.001189
2018-02-08 00:00:00-05:00,-0.051239,-0.027332,-0.085042,-0.045677,-0.048424,-0.045034,-0.047594,-0.037586,-0.054868,-0.043260


,MSFT,AAPL,TSLA,AMZN,NVDA,GOOG,FB,ADBE,NFLX,PYPL
time,,,,,,,,,,
2021-04-21 00:00:00-04:00,0.009021,0.002779,0.034948,0.008049,0.012491,0.000366,-0.004097,-0.003889,-0.073784,0.001626
2021-04-22 00:00:00-04:00,-0.013085,-0.011611,-0.032437,-0.015901,-0.033218,-0.011222,-0.016224,-0.009799,0.001002,-0.009779
2021-04-23 00:00:00-04:00,0.015280,0.017963,0.013365,0.009810,0.027962,0.020891,0.015480,0.017426,-0.007636,0.014336
2021-04-26 00:00:00-04:00,0.001608,0.003127,0.011791,0.020488,0.013986,0.006004,0.006941,-0.000833,0.009594,0.021389
2021-04-27 00:00:00-04:00,0.001797,-0.002524,-0.045068,0.002863,-0.006283,-0.008934,0.000627,0.003180,-0.009464,-0.011114


In [63]:
#Calculate Standard Deviation for stocks and cryptos
stocks_standard_deviation = stocks_daily_df.std()
crypto_standard_deviation = crypto_daily_df.std()
display(stocks_standard_deviation)
display(crypto_standard_deviation)

MSFT    0.020107
AAPL    0.034263
TSLA    0.051178
AMZN    0.020760
NVDA    0.031527
GOOG    0.019560
FB      0.023844
ADBE    0.023089
NFLX    0.027075
PYPL    0.025102
dtype: float64

BTC     0.039263
ETH     0.050653
BNB     0.058845
ADA     0.060871
XRP     0.061960
DOGE    1.808321
LTC     0.054113
LINK    0.073293
BCH     0.064402
XLM     0.062168
dtype: float64

In [64]:
#calculate annual standard deviation for stocks and cryptos
trading_days = 252
stocks_annual_std = stocks_standard_deviation * trading_days ** (1/2)
crypto_annual_std = crypto_standard_deviation * trading_days ** (1/2)
display(stocks_annual_std)
display(crypto_annual_std)

MSFT    0.319191
AAPL    0.543913
TSLA    0.812433
AMZN    0.329555
NVDA    0.500472
GOOG    0.310502
FB      0.378517
ADBE    0.366521
NFLX    0.429799
PYPL    0.398477
dtype: float64

BTC      0.623278
ETH      0.804098
BNB      0.934143
ADA      0.966292
XRP      0.983578
DOGE    28.706212
LTC      0.859022
LINK     1.163498
BCH      1.022346
XLM      0.986881
dtype: float64

In [65]:
# Calculate the annual average return data for stocks and crypto
stocks_annual_avg_return = stocks_daily_df.mean()* trading_days
crypto_annual_avg_return = crypto_daily_df.mean()*trading_days

display(stocks_annual_avg_return)
display(crypto_annual_avg_return)

MSFT    0.368868
AAPL    0.183696
TSLA    0.676082
AMZN    0.334287
NVDA    0.418842
GOOG    0.260254
FB      0.212882
ADBE    0.363417
NFLX    0.292747
PYPL    0.462072
dtype: float64

BTC      0.557221
ETH      0.504919
BNB      1.272034
ADA      0.667469
XRP      0.481963
DOGE    14.967133
LTC      0.466788
LINK     1.556885
BCH      0.407838
XLM      0.440476
dtype: float64

In [66]:
# Calculate the annualized Sharpe Ratios for stocks and crypto
stocks_annual_sharpe_ratio = stocks_annual_avg_return/stocks_annual_std
crypto_annual_sharpe_ratio = crypto_annual_avg_return/crypto_annual_std

display(stocks_annual_sharpe_ratio)
display(crypto_annual_sharpe_ratio)

MSFT    1.155633
AAPL    0.337729
TSLA    0.832169
AMZN    1.014359
NVDA    0.836895
GOOG    0.838170
FB      0.562412
ADBE    0.991529
NFLX    0.681126
PYPL    1.159595
dtype: float64

BTC     0.894017
ETH     0.627932
BNB     1.361712
ADA     0.690752
XRP     0.490010
DOGE    0.521390
LTC     0.543394
LINK    1.338108
BCH     0.398924
XLM     0.446332
dtype: float64

In [80]:
#can't join without tz being the same

#for loop to calculate covariance for each stock to each crypto
covariance_df = pd.DataFrame()
for stock in stock_tickers:
    for crypto in crypto_tickers:
        covariance_df[f'{crypto} cov {stock}'] = crypto_daily_df[crypto].cov(stocks_daily_df[stock])
covariance_df.head()

TypeError: Cannot join tz-naive with tz-aware DatetimeIndex